In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist # mnist dataset is a test dataset already in Keras lib.
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten # "Dropout" is functionality to prevent overfitting. It is modelled as a seperate layer in Keras. "Flatten" is a function which reduces the dimensionality of a tensor.
from keras.layers import Conv2D, MaxPooling2D # "Conv2D" is a 2 dimensional convolutional layer which perfectly fits image data. "MaxPooling2D" is a pooling layer using the max pooling funtion in 2D
from keras import backend as K # K is a function in keras backend.

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data() # We load the data. load_data is good because it returns a tupple of 4 numpy arrays. y_train = target vector, x_train input vector.

if K.image_data_format() == 'channels_first': # This defines the way the colour information is encoded in the image.
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1) # the 1 here signifies its a monochrome image and not a color image.
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # we normalize the data by dividing by 255 to obtain a range between 0 and 1
x_test /= 255
print('x_train shape:', x_train.shape) # We print to verify if the shape is correct.
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes) # The label is recorded as a singular mention with variables between zero and 9, we transform this to "one hot" encoded vector using the Keras to_categorical function
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


10813440/11490434 [===========================>..] - ETA: 0sx_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:


#Please construct the following neural network and report accuracy after training
#Layer 1: 2D Convolution with 32 hidden neurons, kernel 3 by 3, relu activation (Note: input_shape is given)
#Layer 2: 2D Convolution with 64 hidden neurons, kernel 3 by 3, relu activation 
#Layer 3: 2D MaxPooling, pool_size 2 by 2
#Layer 4: 25% Dropout
#Layer 5: Flatten (Hint: model.add(Flatten()))
#Layer 6: Fully connected layer with 128 neurons, relu activation
#Layer 7: 50% Dropout
#Layer 8 Softmax Output Layer according to the problem (output vector)


model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape)) # Layer 1
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu')) # Layer 2
model.add(MaxPooling2D(pool_size = (2,2))) # Layer 3
model.add(Dropout(0.25)) # Layer 4
model.add(Flatten()) # Layer 5 Flatten does 28 * 28 to get 784 neurons
model.add(Dense(128, activation = 'relu')) # Layer 6
model.add(Dropout(0.5)) # Layer 7
model.add(Dense(num_classes, activation = 'softmax')) # Layer 8




In [3]:
# Our loss fn is categorical_crossentropy which is a very good fit for multiclass model task in Keras
# We use an optimizer as Adadelta which turns out to converge best given the dataset that we know the topology 
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

#some learners constantly reported 502 errors in Watson Studio. 
#This is due to the limited resources in the free tier and the heavy resource consumption of Keras.
#This is a workaround to limit resource consumption

from keras import backend as K

K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 386s - loss: 1.8714 - acc: 0.3660 - val_loss: 0.7832 - val_acc: 0.7608
Epoch 2/12
60000/60000 [==============================] - 383s - loss: 1.4104 - acc: 0.5217 - val_loss: 0.7720 - val_acc: 0.8141
Epoch 3/12
60000/60000 [==============================] - 381s - loss: 1.3896 - acc: 0.5405 - val_loss: 0.8451 - val_acc: 0.8013
Epoch 4/12
60000/60000 [==============================] - 380s - loss: 1.3910 - acc: 0.5485 - val_loss: 0.7281 - val_acc: 0.8216
Epoch 5/12
60000/60000 [==============================] - 380s - loss: 1.3565 - acc: 0.5741 - val_loss: 0.8053 - val_acc: 0.8116
Epoch 6/12
60000/60000 [==============================] - 379s - loss: 1.3788 - acc: 0.5787 - val_loss: 1.0700 - val_acc: 0.7941
Epoch 7/12
60000/60000 [==============================] - 380s - loss: 1.3685 - acc: 0.5812 - val_loss: 0.7799 - val_acc: 0.8283
Epoch 8/12
60000/60000 [=======================